In [1]:
import pandas as pd
import numpy as np
import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/edwardlee/Desktop/chromedriver"

In [2]:
browser = webdriver.Chrome(executable_path = chromedriver)

In [3]:
# Get all years for 1996-2016
browser.get('http://stats.nba.com/league/player/')
HTML = browser.page_source
time.sleep(5)
year_data = Selector(text=HTML).xpath("//select[@name='Season']/option/text()").extract()
# all_stats = Selector(text=HTML).xpath("//select[@name='section-links']/option/text()").extract()
time.sleep(5)

# Get all years for gamelogs from 1947-2016
browser.get('http://stats.nba.com/league/player/#!/gamelogs/')
HTML_logs = browser.page_source
time.sleep(5)
logs_year = Selector(text=HTML_logs).xpath("//select[@name='Season']/option/text()").extract()
time.sleep(5)

# Get all play types for category Play Type
# browser.get('http://stats.nba.com/league/player/#!/playtype/')
# HTML_play_type = browser.page_source
# play_type_cats = Selector(text=HTML_play_type).xpath("//div[@class='tabs-wrap']/div[@class='tab']/a/text()").extract()

In [18]:
# Remove first array that says "All Season"
logs_year = logs_year[1:]

In [19]:
# Set parameters
general_para  = ['Base', 'Advanced', 'Misc', 'Scoring', 'Usage', 'Opponent', 'Defense']
clutch_para   = ['Base', 'Advanced', 'Misc', 'Scoring', 'Usage']
shoot_para = ['Base', 'Opponent']
playtype_para = ['Transition', 'Isolation', 'PRBallHandler', 'PRRollman', 'Postup', 'Spotup', 'Handoff', 'Cut', 'OffScreen', 'OffRebound', 'Misc']

In [29]:
## Traditional
# Base
trad_base = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Advanced
trad_adv = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Misc
trad_misc = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Scoring
trad_score = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Usage
trad_usage = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Opponent
trad_opp = 'http://stats.nba.com/stats/leagueplayerondetails?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Defense
trad_def = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

## Clutch
# Base
clutch_base = 'http://stats.nba.com/stats/leaguedashplayerclutch?AheadBehind=Ahead+or+Behind&ClutchTime=Last+5+Minutes&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per48&Period=4&PlayerExperience=&PlayerPosition=&PlusMinus=N&PointDiff=5&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Advanced
clutch_adv = 'http://stats.nba.com/stats/leaguedashplayerclutch?AheadBehind=Ahead+or+Behind&ClutchTime=Last+5+Minutes&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per48&Period=4&PlayerExperience=&PlayerPosition=&PlusMinus=N&PointDiff=5&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Misc
clutch_misc = 'http://stats.nba.com/stats/leaguedashplayerclutch?AheadBehind=Ahead+or+Behind&ClutchTime=Last+5+Minutes&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per48&Period=4&PlayerExperience=&PlayerPosition=&PlusMinus=N&PointDiff=5&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Scoring
clutch_score = 'http://stats.nba.com/stats/leaguedashplayerclutch?AheadBehind=Ahead+or+Behind&ClutchTime=Last+5+Minutes&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per48&Period=4&PlayerExperience=&PlayerPosition=&PlusMinus=N&PointDiff=5&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Usage
clutch_usage = 'http://stats.nba.com/stats/leaguedashplayerclutch?AheadBehind=Ahead+or+Behind&ClutchTime=Last+5+Minutes&College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per48&Period=4&PlayerExperience=&PlayerPosition=&PlusMinus=N&PointDiff=5&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

## Shooting
# Shooting Stats
shooting = 'http://stats.nba.com/stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom=&DateTo=&DistanceRange=5ft+Range&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

# Opponent Shooting Stats
opp_shooting = 'http://stats.nba.com/stats/leaguedashplayershotlocations?College=&Conference=&Country=&DateFrom=&DateTo=&DistanceRange=5ft+Range&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=%s&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

## Player Bios
player_bios = 'http://stats.nba.com/stats/leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

### Tracking Shots (Data fails to load on website)
    
## Game Log, sorted by Date (1946-2015)
game_log = 'http://stats.nba.com/stats/leaguegamelog?Counter=1000&Direction=DESC&LeagueID=00&PlayerOrTeam=P&Season=%s&SeasonType=Regular+Season&Sorter=DATE'

## Play Type (2015-16 only)
play_type = 'http://stats-prod.nba.com/wp-json/statscms/v1/synergy/player/?category=%s&limit=500&name=offensive&q=2456678&season=2015&seasonType=Reg'

## Hustle Stats (2015-16 only)
hustle = 'http://stats.nba.com/stats/leaguehustlestatsplayer?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2015-16&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

In [30]:
# Save all loop-able links into lists to loop later
traditional_stats = [trad_base, trad_adv, trad_misc, trad_score, trad_usage, trad_opp, trad_def]
clutch_stats      = [clutch_base, clutch_adv, clutch_misc, clutch_score, clutch_usage]
shoot_stats       = [shooting, opp_shooting]

In [25]:
browser = webdriver.Chrome(executable_path = chromedriver)
# Traditional Stats, 1996-2016, 7 unique parameters
for trad, stats in zip(traditional_stats, general_para):
    for year in year_data:
        print stats, year
        # Update and save link into variable
        link = trad % (stats, year)
        # Open browser driver through selenium
        browser.get(link)
        # Save selenium page source into variable
        HTML = browser.page_source
        # Save page source into beautifulsoup format
        soup = BeautifulSoup(HTML)
        try:
            # Get all text within <pre> tag and decode from utf-8
            get_text = soup.findAll('pre')[0].getText().encode('utf-8')
            # Load text as json
            json_text = json.loads(get_text)
            # Save json into pandas dataframe
            data = pd.DataFrame(json_text['resultSets'][0]['rowSet'], columns=json_text['resultSets'][0]['headers'])
            # Save to csv
            save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/traditional_stats/%s_%s.csv' % (stats,year)
            data.to_csv(save_csv)
        except:
            print 'Failed', link
        time.sleep(2*np.random.rand())

browser.quit()

Base 2015-16
Base 2014-15
Base 2013-14
Base 2012-13
Base 2011-12
Base 2010-11
Base 2009-10
Base 2008-09
Base 2007-08
Base 2006-07
Base 2005-06
Base 2004-05
Base 2003-04
Base 2002-03
Base 2001-02
Base 2000-01
Base 1999-00
Base 1998-99
Base 1997-98
Base 1996-97
Advanced 2015-16
Advanced 2014-15
Advanced 2013-14
Advanced 2012-13
Advanced 2011-12
Advanced 2010-11
Advanced 2009-10
Advanced 2008-09
Advanced 2007-08
Advanced 2006-07
Advanced 2005-06
Advanced 2004-05
Advanced 2003-04
Advanced 2002-03
Advanced 2001-02
Advanced 2000-01
Advanced 1999-00
Advanced 1998-99
Advanced 1997-98
Advanced 1996-97
Misc 2015-16
Misc 2014-15
Misc 2013-14
Misc 2012-13
Misc 2011-12
Misc 2010-11
Misc 2009-10
Misc 2008-09
Misc 2007-08
Misc 2006-07
Misc 2005-06
Misc 2004-05
Misc 2003-04
Misc 2002-03
Misc 2001-02
Misc 2000-01
Misc 1999-00
Misc 1998-99
Misc 1997-98
Misc 1996-97
Scoring 2015-16
Scoring 2014-15
Scoring 2013-14
Scoring 2012-13
Scoring 2011-12
Scoring 2010-11
Scoring 2009-10
Scoring 2008-09
Scoring 2007

In [31]:
browser = webdriver.Chrome(executable_path = chromedriver)
# Clutch Stats, 1996-2016, 5 unique parameters
for clutch, stats in zip(clutch_stats, clutch_para):
    for year in year_data:
        print stats, year
        # Update and save link into variable
        link = clutch % (stats, year)
        # Open browser driver through selenium
        browser.get(link)
        # Save selenium page source into variable
        HTML = browser.page_source
        # Save page source into beautifulsoup format
        soup = BeautifulSoup(HTML)
        try:
            # Get all text within <pre> tag and decode from utf-8
            get_text = soup.findAll('pre')[0].getText().encode('utf-8')
            # Load text as json
            json_text = json.loads(get_text)
            # Save json into pandas dataframe
            data = pd.DataFrame(json_text['resultSets'][0]['rowSet'], columns=json_text['resultSets'][0]['headers'])
            # Save to csv
            save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/clutch_stats/%s_%s_per48.csv' % (stats,year)
            data.to_csv(save_csv)
        except:
            print 'Failed', link
        time.sleep(2*np.random.rand())

browser.quit()

Base 2015-16
Base 2014-15
Base 2013-14
Base 2012-13
Base 2011-12
Base 2010-11
Base 2009-10
Base 2008-09
Base 2007-08
Base 2006-07
Base 2005-06
Base 2004-05
Base 2003-04
Base 2002-03
Base 2001-02
Base 2000-01
Base 1999-00
Base 1998-99
Base 1997-98
Base 1996-97
Advanced 2015-16
Advanced 2014-15
Advanced 2013-14
Advanced 2012-13
Advanced 2011-12
Advanced 2010-11
Advanced 2009-10
Advanced 2008-09
Advanced 2007-08
Advanced 2006-07
Advanced 2005-06
Advanced 2004-05
Advanced 2003-04
Advanced 2002-03
Advanced 2001-02
Advanced 2000-01
Advanced 1999-00
Advanced 1998-99
Advanced 1997-98
Advanced 1996-97
Misc 2015-16
Misc 2014-15
Misc 2013-14
Misc 2012-13
Misc 2011-12
Misc 2010-11
Misc 2009-10
Misc 2008-09
Misc 2007-08
Misc 2006-07
Misc 2005-06
Misc 2004-05
Misc 2003-04
Misc 2002-03
Misc 2001-02
Misc 2000-01
Misc 1999-00
Misc 1998-99
Misc 1997-98
Misc 1996-97
Scoring 2015-16
Scoring 2014-15
Scoring 2013-14
Scoring 2012-13
Scoring 2011-12
Scoring 2010-11
Scoring 2009-10
Scoring 2008-09
Scoring 2007

In [27]:
browser = webdriver.Chrome(executable_path = chromedriver)
# Shooting Stats, 1996-2016, 2 unique parameters
for shoot, stats in zip(shoot_stats, shoot_para):
    for year in year_data:
        print stats, year
        # Update and save link into variable
        link = shoot % (stats, year)
        # Open browser driver through selenium
        browser.get(link)
        # Save selenium page source into variable
        HTML = browser.page_source
        # Save page source into beautifulsoup format
        soup = BeautifulSoup(HTML)
        try:
            # Get all text within <pre> tag and decode from utf-8
            get_text = soup.findAll('pre')[0].getText().encode('utf-8')
            # Load text as json
            json_text = json.loads(get_text)
            # Create custom headers for each dataframe
            distance_col = json_text['resultSets']['headers'][0]['columnNames']
            distance_col = [col.replace('.','').replace(' ','_').lower() for col in distance_col]
            tmp = len(json_text['resultSets']['headers'][1]['columnNames'][:5])
            tmp1 = len(json_text['resultSets']['headers'][1]['columnNames'][6:])
            new_cols = []

            for i in range(tmp):
                new_cols.append(json_text['resultSets']['headers'][1]['columnNames'][i])

            for col in distance_col:
                for i in range(tmp, tmp1):
                    save_tmp = json_text['resultSets']['headers'][1]['columnNames'][i]
                    name_replace = save_tmp + '_' + col
                    new_cols.append(name_replace)
                    if save_tmp == 'FG_PCT':
                        break
                    elif save_tmp == 'OPP_FG_PCT':
                        break
            # Save json into pandas dataframe
            data = pd.DataFrame(json_text['resultSets']['rowSet'], columns=new_cols)
            # Save to csv
            save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/shooting_stats/%s_%s.csv' % (stats,year)
            data.to_csv(save_csv)
        except:
            print 'Failed', link

        time.sleep(2*np.random.rand())
browser.quit()

Base 2015-16
Base 2014-15
Base 2013-14
Base 2012-13
Base 2011-12
Base 2010-11
Base 2009-10
Base 2008-09
Base 2007-08
Base 2006-07
Base 2005-06
Base 2004-05
Base 2003-04
Base 2002-03
Base 2001-02
Base 2000-01
Base 1999-00
Base 1998-99
Base 1997-98
Base 1996-97
Opponent 2015-16
Opponent 2014-15
Opponent 2013-14
Opponent 2012-13
Opponent 2011-12
Opponent 2010-11
Opponent 2009-10
Opponent 2008-09
Opponent 2007-08
Opponent 2006-07
Opponent 2005-06
Opponent 2004-05
Opponent 2003-04
Opponent 2002-03
Opponent 2001-02
Opponent 2000-01
Opponent 1999-00
Opponent 1998-99
Opponent 1997-98
Opponent 1996-97


In [28]:
browser = webdriver.Chrome(executable_path = chromedriver)
# Player bios, 1996-2016, no unique parameters
for year in year_data:
    print year
    # Update and save link into variable
    link = player_bios % year
    # Open browser driver through selenium
    browser.get(link)
    # Save selenium page source into variable
    HTML = browser.page_source
    # Save page source into beautifulsoup format
    soup = BeautifulSoup(HTML)
    try:
        # Get all text within <pre> tag and decode from utf-8
        get_text = soup.findAll('pre')[0].getText().encode('utf-8')
        # Load text as json
        json_text = json.loads(get_text)
        # Save json into pandas dataframe
        data = pd.DataFrame(json_text['resultSets'][0]['rowSet'], columns=json_text['resultSets'][0]['headers'])
        # Save to csv
        save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/player_bios/%s_%s.csv' % ('bios',year)
        data.to_csv(save_csv)
    except:
        print 'Failed', link
    time.sleep(2*np.random.rand())

browser.quit()

2015-16
2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97


In [27]:
browser = webdriver.Chrome(executable_path = chromedriver)
# Hustle Stats, 1 season, no unique parameters

# Update and save link into variable
link = hustle
# Open browser driver through selenium
browser.get(link)
# Save selenium page source into variable
HTML = browser.page_source
# Save page source into beautifulsoup format
soup = BeautifulSoup(HTML)
# Get all text within <pre> tag and decode from utf-8
get_text = soup.findAll('pre')[0].getText().encode('utf-8')
# Load text as json
json_text = json.loads(get_text)
# Save json into pandas dataframe
data = pd.DataFrame(json_text['resultSets'][0]['rowSet'], columns=json_text['resultSets'][0]['headers'])
# Save to csv
save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/other_stats/%s_%s.csv' % ('hustle','2015-16')
data.to_csv(save_csv)

browser.quit()

In [28]:
browser = webdriver.Chrome(executable_path = chromedriver)
# Game Logs per Season, 1946-2016, no unique parameters
for year in logs_year:
    print year
    # Update and save link into variable
    link = game_log % year
    # Open browser driver through selenium
    browser.get(link)
    # Save selenium page source into variable
    HTML = browser.page_source
    # Save page source into beautifulsoup format
    soup = BeautifulSoup(HTML)
    try:
        # Get all text within <pre> tag and decode from utf-8
        get_text = soup.findAll('pre')[0].getText().encode('utf-8')
        # Load text as json
        json_text = json.loads(get_text)
        # Save json into pandas dataframe
        data = pd.DataFrame(json_text['resultSets'][0]['rowSet'], columns=json_text['resultSets'][0]['headers'])
        # Save to csv
        save_csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/datasets/gamelogs/%s_%s.csv' % ('gamelogs',year)
        data.to_csv(save_csv)
    except:
        print 'Failed', link
    time.sleep(5*np.random.rand())    

All Seasons
Failed http://stats.nba.com/stats/leaguegamelog?Counter=1000&Direction=DESC&LeagueID=00&PlayerOrTeam=P&Season=All Seasons&SeasonType=Regular+Season&Sorter=DATE
2015-16
2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97
1995-96
1994-95
1993-94
1992-93
1991-92
1990-91
1989-90
1988-89
1987-88
1986-87
1985-86
1984-85
1983-84
1982-83
1981-82
1980-81
1979-80
1978-79
1977-78
1976-77
1975-76
1974-75
1973-74
1972-73
1971-72
1970-71
1969-70
1968-69
1967-68
1966-67
1965-66
1964-65
1963-64
1962-63
1961-62
1960-61
1959-60
1958-59
1957-58
1956-57
1955-56
1954-55
1953-54
1952-53
1951-52
1950-51
1949-50
1948-49
1947-48
1946-47


# Scrap Yahoo, ESPN, CBS 2016-17 Preseason Ranking

In [14]:
# ESPN Preseason Ranking
rank_list = []
playername_list = []
for startIndex in range(0, 680, 40):
    url_format = "http://games.espn.com/fba/tools/projections?startIndex=%d" % startIndex
    r = requests.get(url_format)
    
    HTML = r.text
    rank_data = Selector(text=HTML).xpath("//td[@class='playertableData']/text()").extract()
    rank_list += rank_data
    playername_data = Selector(text=HTML).xpath("//td[@class='playertablePlayerName']//a//text()").extract()
    playername_list += playername_data
    time.sleep(2*np.random.rand())

print 'Scrap Completed'

Scrap Completed


In [22]:
rank_list = filter(lambda a: a != 'RNK', rank_list)

In [23]:
len(rank_list), len(playername_list)

(680, 680)

In [26]:
espn_ranking = pd.DataFrame({'ESPNRank_16-17':rank_list, 'PLAYER_NAME': playername_list})

In [47]:
espn_ranking.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/comparison_dataset/2016-17_espn_rank.csv')

In [29]:
# CBS Preseason Ranking
rank_list = []
playername_list = []

url_format = "http://www.cbssports.com/fantasy/basketball/draft/averages?&print_rows=9999"
r = requests.get(url_format)

HTML = r.text
rank_data = Selector(text=HTML).xpath("//tr[@class='row1']/td[1]/text()").extract()
rank_data_2 = Selector(text=HTML).xpath("//tr[@class='row2']/td[1]/text()").extract()
rank_list += rank_data
rank_list += rank_data_2
playername_data = Selector(text=HTML).xpath("//tr[@class='row1']/td[2]/a/text()").extract()
playername_data_2 = Selector(text=HTML).xpath("//tr[@class='row2']/td[2]/a/text()").extract()
playername_list += playername_data
playername_list += playername_data_2
time.sleep(2*np.random.rand())

print 'Scrap Completed'

Scrap Completed


In [31]:
len(rank_list), len(playername_list)

(230, 230)

In [32]:
cbs_ranking = pd.DataFrame({'CBSRank_16-17':rank_list, 'PLAYER_NAME': playername_list})

In [38]:
cbs_ranking['CBSRank_16-17'] = cbs_ranking['CBSRank_16-17'].astype(int)

In [42]:
cbs_ranking = cbs_ranking.sort_values('CBSRank_16-17', ascending=True).reset_index(drop=True)

In [45]:
cbs_ranking.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/comparison_dataset/2016-17_cbs_rank.csv')

In [ ]:
# import requests

# url = "http://stats.nba.com/league/team/#!/advanced/"
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.99 Safari/537.36'}

# with requests.Session() as session:
#     session.headers = headers
#     session.get(url, headers=headers)

#     params = {
#         'DateFrom': '',
#         'DateTo': '',
#         'GameScope': '',
#         'GameSegment': '',
#         'LastNGames': '0',
#         'LeagueID': '00',
#         'Location': '',
#         'MeasureType': 'Advanced',
#         'Month': '0',
#         'OpponentTeamID': '0',
#         'Outcome': '',
#         'PaceAdjust': 'N',
#         'PerMode': 'Totals',
#         'Period': '0',
#         'PlayerExperience': '',
#         'PlayerPosition': '',
#         'PlusMinus': 'N',
#         'Rank': 'N',
#         'Season': '2014-15',
#         'SeasonSegment': '',
#         'SeasonType': 'Regular Season',
#         'StarterBench': '',
#         'VsConference': '',
#         'VsDivision': ''
#     }

#     response = session.get('http://stats.nba.com/stats/leaguedashteamstats', params=params)
#     results = response.json()
#     headers = results['resultSets'][0]['headers']
#     rows = results['resultSets'][0]['rowSet']
#     for row in rows:
#         print(dict(zip(headers, row)))